# Scrape the NASA Website

In [1]:
#import dependencies

#beautiful soup
from bs4 import BeautifulSoup as bs

#requests
import requests

#splinter
from splinter import Browser

#pandas
import pandas as pd

#selenium???
from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
import time

In [5]:
#define URLs for our various news sources

url_news = "https://mars.nasa.gov/news/"
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_weather = "https://twitter.com/marswxreport?lang=en"
url_facts = "https://space-facts.com/mars/"
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_chicago = "https://forecast.weather.gov/MapClick.php?lat=42.012590000000046&lon=-87.68613999999997#.XlKRDShKiUl"

In [ ]:
#Pull the information for NASA's Mars News.
    #the div class for the headlines is "content_title"
    #the div class for the paragraph is "rollover_description_inner"

response = requests.get(url_news)

soup = bs(response.text, 'html.parser')

print(soup.prettify())



In [ ]:
#store the headline and text as a variable

news_headline = soup.find('div', class_="content_title").text
news_headline = news_headline.replace("\n","")

news_headline

In [ ]:
news_byline = soup.find('div', class_="rollover_description_inner").text
news_byline = news_byline.replace("\n","")
news_byline

In [ ]:
# response = requests.get(url_image)
# soup = bs(response.text, 'html.parser')

# print(soup.prettify())

In [ ]:
#Pull the JPL Mars Image URL (using Splinter per directions in the ReadMe)

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url_image)

html = browser.html
soup = bs(html, 'html.parser')

slide = soup.find('li', class_="slide")
fancybox = slide.find('a')
fimage_name = fancybox['data-title']
fimage_desc = fancybox['data-description']
fimage_purl = fancybox['data-fancybox-href']

fimage_url = f"https://www.jpl.nasa.gov{fimage_purl}"

browser.quit()

print(fimage_name)
print(fimage_desc)
print(fimage_url)


In [7]:
#pull the martian weather report from Twitter to find out whether Chicago is colder than mars right now
    #(it happens.)

    #selenium??? 
driver = webdriver.Chrome()  #https://stackoverflow.com/questions/59222947/retrieving-text-from-twitter-account-using-beautifulsoup-and-splinter
driver.get(url_weather)

#driver.wait = WebDriverWait(driver,5)  #https://allofyourbases.com/2018/01/16/mining-twitter-with-selenium/

time.sleep(15)

html = driver.page_source

driver.close()

soup = bs(html, 'html.parser')

weather = soup.find('div', class_='css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0').text

weather


'InSight sol 441 (2020-02-22) low -95.1ºC (-139.1ºF) high -13.5ºC (7.7ºF)\nwinds from the SSE at 6.2 m/s (13.8 mph) gusting to 21.0 m/s (47.0 mph)\npressure at 6.30 hPa'

In [12]:
#chicago weather - high temp is...

response = requests.get(url_chicago)

soup = bs(response.text, 'html.parser')

chicago_weather = soup.find('p', class_="myforecast-current-lrg").text

chiberia = chicago_weather.replace("°F","")

chiberia = int(chiberia)

chiberia

37

In [20]:
#mars weather - high temp is...

mars_weather = weather.split(")")
mars_weather = mars_weather[2]
mars_weather_2 = mars_weather.split("(")
mars_weather_3 = mars_weather_2[1].replace("ºF","")
martian_temp = float(mars_weather_3)
martian_temp = int(martian_temp)
martian_temp

7

In [ ]:
#is it actually colder on Mars today? 

if martian_temp > chiberia:
    ismarscolder = False
else:
    ismarscolder = True

In [10]:
#pull the list of mars facts from the mars facts website

facts = pd.read_html(url_facts)
facts = facts[0]

facts.columns = ['Datum', 'Value']

facts = facts.set_index('Datum')

facts

,Value
Datum,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
facts_html = facts.to_html()

facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Datum</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </t

In [21]:
#here come the four martian hemispheres

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
    
hemisphere_urls = []
    
browser.visit(url_hemispheres)

html = browser.html
soup = bs(html, 'html.parser')

results = soup.find('div', class_='container').find_all('h3')

for result in results:
    
    temp_dict = {}
    
    print(result)
    title = str(result)
    title = title.replace("<h3>","")
    title = title.replace("</h3>","")
    print(title)
    
    browser.click_link_by_partial_text(title)
  
    html = browser.html
    soup = bs(html, 'html.parser')

    spot = soup.find('div', class_='container')
    image = spot.find('img', class_='wide-image')
    url = image['src']
    url = f"https://astrogeology.usgs.gov{url}"
    print(url)
    
    key1 = "title"
    key2 = "img_url"
    
    temp_dict.update({key1: title, key2: url})
    
    hemisphere_urls.append(temp_dict)
    
    browser.back()
    
browser.quit()
print(hemisphere_urls)

<h3>Cerberus Hemisphere Enhanced</h3>
Cerberus Hemisphere Enhanced


C:\ProgramData\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
<h3>Schiaparelli Hemisphere Enhanced</h3>
Schiaparelli Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
<h3>Syrtis Major Hemisphere Enhanced</h3>
Syrtis Major Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
<h3>Valles Marineris Hemisphere Enhanced</h3>
Valles Marineris Hemisphere Enhanced
astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere

In [ ]:
"""
To review:
Mars News Headline: news_headline
Mars News Byline: news_byline

Mars Image Name: fimage_name
Mars Image Description: fimage_desc
Mars Image URL: fimage_url

Mars Weather Report: weather
Chicago High Temperature: chiberia
Martian High Temperature: martian_temp
Binary Temperature Variable: ismarscolder

Facts List: facts
Facts HTML Code: facts_html

Hemispheres URL Dictionary: hemisphere_urls

"""